# CS452/CS552 Assignment 2: Car Rollover Prediction

**Release Date: 23.11.2021** <br>
**Submission Deadline: 12.12.2021 23.55**

In [1]:
# Author: Aylin Aydın
# Department: Computer Science
# Degree: BSc.

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import requests as rq
import io
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score, fbeta_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.inspection import permutation_importance

from sklearn.naive_bayes import GaussianNB, CategoricalNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 150)
random.seed(42) # DO NOT CHANGE
np.random.seed(42) # DO NOT CHANGE

# Part 1: Data Loading and Cleaning
> Get FARS datasets through the API provided by NHTSA using requests library.<br>

Two types of datasets will be used in the project:<br>
*person* dataset provides the information of persons involved in a crash.<br>
*vindecode* dataset the information of vehicles involved in a crash thanks to decoded Vehicle Identification Number (VIN).

In [3]:
api = "https://crashviewer.nhtsa.dot.gov/CrashAPI/FARSData/GetFARSData"
case_years = np.arange(2014, 2020, dtype=int) # DO NOT CHANGE

In [4]:
def get_persons_data(year):
    response = rq.get(url = api, params = {'dataset':'person', 'caseYear':year, 'format':'csv'})

    if response.ok:
        csv_data = response.content
        persons = pd.read_csv(io.StringIO(csv_data.decode('utf-8')))
        return persons
    
    print("Failed to get the data")
    print(response.status)
    
    return None

def get_vehicles_data(year):
    response = rq.get(url = api, params = {'dataset':'vindecode', 'caseYear':year, 'format':'csv'})

    if response.ok:
        csv_data = response.content
        vehicles = pd.read_csv(io.StringIO(csv_data.decode('utf-8')))
        return vehicles
    
    print("Failed to get the data")
    print(response.status)
    
    return None

1.1) Use the above methods to get vehicles and persons datasets from 2014 to 2019.

In [5]:
p_2014=get_persons_data(2014)
p_2015=get_persons_data(2015)
p_2016=get_persons_data(2016)
p_2017=get_persons_data(2017)
p_2018=get_persons_data(2018)
p_2019=get_persons_data(2019)
    

persons=pd.concat([p_2014,p_2015,p_2016,p_2017,p_2018,p_2019], axis=0)

C:\Users\aylin.LAPTOP-TJ9MCDMK\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\aylin.LAPTOP-TJ9MCDMK\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (17,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
_2014=get_vehicles_data(2014)
_2015=get_vehicles_data(2015)
_2016=get_vehicles_data(2016)
_2017=get_vehicles_data(2017)
_2018=get_vehicles_data(2018)
_2019=get_vehicles_data(2019)
    

vehicles=pd.concat([_2014,_2015,_2016,_2017,_2018,_2019], axis=0)

C:\Users\aylin.LAPTOP-TJ9MCDMK\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (25,51,52,96) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\aylin.LAPTOP-TJ9MCDMK\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\aylin.LAPTOP-TJ9MCDMK\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\aylin.LAPTOP-TJ9MCDMK\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, re

1.2) Determine the useless columns to drop from the dataframes.

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
vehicles

,caseyear,state,statename,st_case,veh_no,ncicmake,vinyear,vehtype,vehtype_t,vinmake_t,vinmodel_t,vintrim_t,vintrim1_t,vintrim2_t,vintrim3_t,vintrim4_t,bodystyl,bodystyl_t,doors,wheels,drivwhls,mfg,mfg_t,displci,displcc,cylndrs,cycles,fuel,fuel_t,fuelinj,fuelinj_t,carbtype,carbtype_t,carbbrls,gvwrange,gvwrange_t,whlbsh,whlblg,tiredesc_f,psi_f,tiresz_f,tiresz_f_t,tiredesc_r,psi_r,rearsize,rearsize_t,tonrating,shipweight,msrp,drivetyp,drivetyp_t,salectry,salectry_t,abs,abs_t,security,security_t,drl,drl_t,rstrnt,rstrnt_t,tkcab,tkcab_t,tkaxlef,tkaxlef_t,tkaxler,tkaxler_t,tkbrak,tkbrak_t,engmfg,engmfg_t,engmodel,tkduty,tkduty_t,tkbedl,tkbedl_t,segmnt,segmnt_t,plant,plntctry_t,plntcity,plntctry,plntstat,plntstat_t,origin,origin_t,dispclmt,blocktype,enghead,enghead_t,vlvclndr,vlvtotal,engvincd,incomplt,battyp,battyp_t,batkwrtg,batvolt,supchrgr,supchrgr_t,turbo,turbo_t,engvvt,mcyusage,mcyusage_t
0,2014,1,Alabama,10001,1,TOYT,2011.0,P,Passenger Car,TOYOTA,COROLLA,BASE,S,LE,NaN,NaN,SD,Sedan,4.0,4.0,2.0,C175,TOYOTA,110.0,0.0,4.0,NaN,G,Gas,NaN,NaN,F,Fuel Injection,NaN,NaN,NaN,102.4,102.4,NaN,0.0,29,15R195,NaN,0.0,29.0,15R195,NaN,2734.0,15600.0,FWD,Front Wheel Drive,NaN,NaN,2,All Wheel Std,F,Immobilizer and keyless entry,S,Standard,7,Du Frnt/Sd/Hd Air Bgs/Rr Hd Ar Bgs/Act Belts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,Non Luxury Traditional Compact,C,Canada,CAMBRIDGE,CAN,ON,ONTARIO,B,Import Built in North America,1.8,In-Line,DOHC,Double Overhead Camshaft,4.0,16.0,U,N,NaN,NaN,NaN,NaN,N,No,N,No,NaN,NaN,NaN
1,2014,1,Alabama,10002,1,DODG,1997.0,T,Truck,DODGE,RAM 2500,NaN,NaN,NaN,NaN,NaN,PK,Pickup,2.0,4.0,2.0,C127,DAIMLER-CHRYSLER,360.0,0.0,8.0,NaN,G,Gas,U,Unknown,F,Fuel Injection,NaN,2.0,"6,001 - 10,000#",138.7,154.7,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,C,4787.0,20775.0,RWD,Rear Wheel Drive,NaN,NaN,4,Other Std,U,Unknown,N,Not Available,M,Driver Front Air Bag/Passenger Active Belts,EXT,Extended Cab,N,Standard Axle,S,Single,HYD,HYDRAULIC,050,CHRYSLER,5.9L,ME,Medium Duty,R,Regular,7,Non Luxury Full Size 3qtr to 1 Ton Pickup,M,Mexico,LAGO ALBERTO,MEX,NaN,NaN,D,Domestic,5.9,V-type,OHV,Overhead Valve,2.0,16.0,Z,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,1,Alabama,10003,1,CHEV,2004.0,P,Passenger Car,CHEVROLET,MALIBU,LT,NaN,NaN,NaN,NaN,SD,Sedan,4.0,0.0,0.0,C137,GENERAL MOTORS,214.0,0.0,6.0,NaN,G,Gas,U,Unknown,F,Fuel Injection,NaN,NaN,NaN,106.3,106.3,NaN,0.0,38,16R215,NaN,0.0,NaN,NaN,NaN,3315.0,23495.0,FWD,Front Wheel Drive,NaN,NaN,2,All Wheel Std,P,Pass Key,S,Standard,K,Dual Air Bags Front and Sides/Active Belts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,Non Luxury Traditional Mid Size,F,United States,FAIRFAX,USA,KS,KANSAS,D,Domestic,3.5,V-type,NaN,NaN,0.0,0.0,8,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,1,Alabama,10003,2,TOYT,1997.0,P,Passenger Car,TOYOTA,CAMRY,CE,LE,XLE,NaN,NaN,SD,Sedan,4.0,0.0,0.0,C175,TOYOTA,183.0,0.0,6.0,NaN,G,Gas,U,Unknown,F,Fuel Injection,NaN,NaN,NaN,105.1,105.1,NaN,0.0,30,15R205,NaN,0.0,NaN,NaN,NaN,3086.0,19248.0,FWD,Front Wheel Drive,NaN,NaN,5,All Wheel Opt,U,Unknown,N,Not Available,E,Dual Front Air Bag/Active Belts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,Non Luxury Traditional Mid Size,U,United States,GEORGETOWN,USA,KY,KENTUCKY,B,Import Built in North America,3.0,V-type,NaN,NaN,4.0,24.0,F,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,1,Alabama,10004,1,TOYT,1999.0,T,Truck,TOYOTA,TACOMA,NaN,NaN,NaN,NaN,NaN,PK,Pickup,2.0,4.0,2.0,C175,TOYOTA,149.0,0.0,4.0,NaN,G,Gas,U,Unknown,F,Fuel Injection,NaN,1.0,"0 - 6,000#",103.3,103.3,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,B,2567.0,12698.0,RWD,Rear Wheel Drive,NaN,NaN,5,All Wheel Opt,U,Unknown,N,Not Available,Z,Du Ar Bgs Frnt/Act Blts/Pass Deactivate/cutoff,CON,Conventional,U,Unknown,U,Unknown,U,Unknown,NaN,NaN,NaN,U,Unknown,R,Regular,T,Non Luxury Mid Size Pickup,Z,United States,FREMONT,USA,CA,CALIFORNIA,B,Import Built in North America,2.4,In-Line,DOHC,Double Overhead Camshaft,0.0,0.0,L,N,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [9]:
persons

,caseyear,state,statename,st_case,ve_forms,veh_no,per_no,str_veh,str_vehname,county,countyname,day,month,monthname,hour,hourname,minute,minutename,road_fnc,road_fncname,harm_ev,harm_evname,man_coll,man_collname,sch_bus,sch_busname,make,makename,mak_mod,body_typ,body_typname,mod_year,mod_yearname,tow_veh,tow_vehname,spec_use,spec_usename,emer_use,emer_usename,rollover,rollovername,impact1,impact1name,fire_exp,fire_expname,age,agename,sex,sexname,per_typ,per_typname,inj_sev,inj_sevname,seat_pos,seat_posname,rest_use,rest_usename,rest_mis,rest_misname,air_bag,air_bagname,ejection,ejectionname,ej_path,ej_pathname,extricat,extricatname,drinking,drinkingname,alc_det,alc_detname,alc_status,alc_statusname,atst_typ,atst_typname,alc_res,alc_resname,drugs,drugsname,drug_det,drug_detname,dstatus,dstatusname,drugtst1,drugtst1name,drugres1,drugres1name,drugtst2,drugtst2name,drugres2,drugres2name,drugtst3,drugtst3name,drugres3,drugres3name,hospital,hospitalname,doa,doaname,death_da,death_daname,death_mo,death_moname,death_yr,death_yrname,death_hr,death_hrname,death_mn,death_mnname,death_tm,lag_hrs,lag_mins,p_sf1,p_sf1name,p_sf2,p_sf2name,p_sf3,p_sf3name,cert_no,work_inj,work_injname,hispanic,hispanicname,race,racename,location,locationname,func_sys,func_sysname,rur_urb,rur_urbname
0,2014,1,Alabama,10001,1,1,1,0,Occupant of a Motor Vehicle,71,JACKSON (71),1,1,January,1,1:00am-1:59am,15,15,5.0,Rural-Minor Collector,17,Boulder,0,Not a Collision with Motor Vehicle In-Transport,0,No,49.0,Toyota,49032.0,4.0,"4-door sedan, hardtop",2011.0,2011,0.0,No Trailing Units,0.0,No Special Use,0.0,Not Applicable,1.0,"Rollover, Tripped by Object/Vehicle",11.0,11 Clock Point,0.0,No or Not Reported,24,24 Years,1,Male,1,Driver of a Motor Vehicle In-Transport,4,Fatal Injury (K),11,"Front Seat, Left Side",3,Shoulder and Lap Belt Used,0,No,8,Deployed- Combination,0,Not Ejected,0,Not Ejected/Not Applicable,1,Extricated,1,Yes (Alcohol Involved),5,Observed,2,Test Given,1,Blood,26,0.26 % BAC,9,Unknown,8,Not Reported,2,Test Given,1.0,Blood,998.0,"Tested For Drugs, Drugs Found, Type unknown/Po...",0.0,Test Not Given,0.0,Test Not Given,0.0,Test Not Given,0.0,Test Not Given,0,Not Transported,7,Died at Scene,1,1,1,January,2014,2014,1,1:00-1:59,15,15,115,0,0,0,None,0,None,0,None,************,0,No,7,Non-Hispanic,1.0,White,0,0,NaN,NaN,NaN,NaN
1,2014,1,Alabama,10001,1,1,2,0,Occupant of a Motor Vehicle,71,JACKSON (71),1,1,January,1,1:00am-1:59am,15,15,5.0,Rural-Minor Collector,17,Boulder,0,Not a Collision with Motor Vehicle In-Transport,0,No,49.0,Toyota,49032.0,4.0,"4-door sedan, hardtop",2011.0,2011,0.0,No Trailing Units,0.0,No Special Use,0.0,Not Applicable,1.0,"Rollover, Tripped by Object/Vehicle",11.0,11 Clock Point,0.0,No or Not Reported,30,30 Years,2,Female,2,Passenger of a Motor Vehicle In-Transport,2,Suspected Minor Injury(B),13,"Front Seat, Right Side",3,Shoulder and Lap Belt Used,0,No,8,Deployed- Combination,0,Not Ejected,0,Not Ejected/Not Applicable,0,Not Extricated or Not Applicable,8,Not Reported,9,Not Reported,0,Test Not Given,0,Test Not Given,96,Test Not Given,8,Not Reported,8,Not Reported,0,Test Not Given,0.0,Test Not Given,0.0,Test Not Given,0.0,Test Not Given,0.0,Test Not Given,0.0,Test Not Given,0.0,Test Not Given,1,EMS Air,0,Not Applicable,88,Not Applicable (Non-Fatal),88,Not Applicable (Non-Fatal),8888,Not Applicable (Non-fatal),88,Not Applicable (Non-fatal),88,Not Applicable (Non-fatal),8888,999,99,0,None,0,None,0,None,************,8,Not Applicable (not a fatality),0,Not A Fatality (not Applicable),0.0,Not a Fatality (not Applicable),0,0,NaN,NaN,NaN,NaN
2,2014,1,Alabama,10002,1,1,1,0,Occupant of a Motor Vehicle,59,FRANKLIN (59),1,1,January,13,1:00pm-1:59pm,30,30,6.0,Rural-Local Road or Street,42,Tree (Standing Only),0,Not a Collision with Motor Vehicle In-Transport,0,No,7.0,Dodge,7482.0,31.0,"Standard pickup (GVWR 4,500 to 10,00 lbs.)(Jee...",1997.0,1997,0.0,No Trailing Units,0.0,No Special Use,0.0,Not Applicable,0.0,No Rollover,12.0,12 Clock Point,0.0,No o

In [10]:
persons["str_vehname"].values

array(['Occupant of a Motor Vehicle', 'Occupant of a Motor Vehicle',
       'Occupant of a Motor Vehicle', ..., 'Occupant of a Motor Vehicle',
       'Occupant of a Motor Vehicle', 'Occupant of a Motor Vehicle'],
      dtype=object)

In [11]:
copy_persons=persons.copy()
copy_vehicles=vehicles.copy()

In [12]:
copy2_persons=persons.copy()
copy2_vehicles=vehicles.copy()

In [13]:
vehicles.columns

Index(['caseyear', 'state', 'statename', 'st_case', 'veh_no', 'ncicmake',
       'vinyear', 'vehtype', 'vehtype_t', 'vinmake_t',
       ...
       'battyp_t', 'batkwrtg', 'batvolt', 'supchrgr', 'supchrgr_t', 'turbo',
       'turbo_t', 'engvvt', 'mcyusage', 'mcyusage_t'],
      dtype='object', length=105)

In [14]:
persons["hourname"].unique()

array(['1:00am-1:59am', '1:00pm-1:59pm', '3:00am-3:59am', '9:00am-9:59am',
       '4:00pm-4:59pm', '2:00am-2:59am', '6:00pm-6:59pm', '6:00am-6:59am',
       '3:00pm-3:59pm', '5:00pm-5:59pm', '8:00am-8:59am',
       '12:00pm-12:59pm', '9:00pm-9:59pm', '8:00pm-8:59pm',
       '10:00am-10:59am', '7:00am-7:59am', '10:00pm-10:59pm',
       '7:00pm-7:59pm', '2:00pm-2:59pm', '11:00am-11:59am',
       '0:00am-0:59am', '5:00am-5:59am', '11:00pm-11:59pm',
       '4:00am-4:59am', 'Unknown Hours'], dtype=object)

In [15]:
persons["hour"].unique()

array([ 1, 13,  3,  9, 16,  2, 18,  6, 15, 17,  8, 12, 21, 20, 10,  7, 22,
       19, 14, 11,  0,  5, 23,  4, 99], dtype=int64)

In [16]:
vehicles.columns[5]

'ncicmake'

In [17]:
vehicles["carbtype_t"].unique()

array(['Fuel Injection', nan, 'Carburetor', 'Electric (n/a)', 'Unknown',
       'FUEL INJECTION'], dtype=object)

In [18]:
#for i in range(105):
    #print(vehicles.columns[i], ": ",vehicles.isnull().sum()[i])

In [19]:
#for i in range(131):
    #print(persons.columns[i], ": ",persons.isnull().sum()[i])

In [20]:
#cc to cubic inch 
copy_vehicles["displci"]=copy_vehicles["displci"]+(copy_vehicles["displcc"]/16.4)
copy_vehicles

,caseyear,state,statename,st_case,veh_no,ncicmake,vinyear,vehtype,vehtype_t,vinmake_t,vinmodel_t,vintrim_t,vintrim1_t,vintrim2_t,vintrim3_t,vintrim4_t,bodystyl,bodystyl_t,doors,wheels,drivwhls,mfg,mfg_t,displci,displcc,cylndrs,cycles,fuel,fuel_t,fuelinj,fuelinj_t,carbtype,carbtype_t,carbbrls,gvwrange,gvwrange_t,whlbsh,whlblg,tiredesc_f,psi_f,tiresz_f,tiresz_f_t,tiredesc_r,psi_r,rearsize,rearsize_t,tonrating,shipweight,msrp,drivetyp,drivetyp_t,salectry,salectry_t,abs,abs_t,security,security_t,drl,drl_t,rstrnt,rstrnt_t,tkcab,tkcab_t,tkaxlef,tkaxlef_t,tkaxler,tkaxler_t,tkbrak,tkbrak_t,engmfg,engmfg_t,engmodel,tkduty,tkduty_t,tkbedl,tkbedl_t,segmnt,segmnt_t,plant,plntctry_t,plntcity,plntctry,plntstat,plntstat_t,origin,origin_t,dispclmt,blocktype,enghead,enghead_t,vlvclndr,vlvtotal,engvincd,incomplt,battyp,battyp_t,batkwrtg,batvolt,supchrgr,supchrgr_t,turbo,turbo_t,engvvt,mcyusage,mcyusage_t
0,2014,1,Alabama,10001,1,TOYT,2011.0,P,Passenger Car,TOYOTA,COROLLA,BASE,S,LE,NaN,NaN,SD,Sedan,4.0,4.0,2.0,C175,TOYOTA,110.000000,0.0,4.0,NaN,G,Gas,NaN,NaN,F,Fuel Injection,NaN,NaN,NaN,102.4,102.4,NaN,0.0,29,15R195,NaN,0.0,29.0,15R195,NaN,2734.0,15600.0,FWD,Front Wheel Drive,NaN,NaN,2,All Wheel Std,F,Immobilizer and keyless entry,S,Standard,7,Du Frnt/Sd/Hd Air Bgs/Rr Hd Ar Bgs/Act Belts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,Non Luxury Traditional Compact,C,Canada,CAMBRIDGE,CAN,ON,ONTARIO,B,Import Built in North America,1.8,In-Line,DOHC,Double Overhead Camshaft,4.0,16.0,U,N,NaN,NaN,NaN,NaN,N,No,N,No,NaN,NaN,NaN
1,2014,1,Alabama,10002,1,DODG,1997.0,T,Truck,DODGE,RAM 2500,NaN,NaN,NaN,NaN,NaN,PK,Pickup,2.0,4.0,2.0,C127,DAIMLER-CHRYSLER,360.000000,0.0,8.0,NaN,G,Gas,U,Unknown,F,Fuel Injection,NaN,2.0,"6,001 - 10,000#",138.7,154.7,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,C,4787.0,20775.0,RWD,Rear Wheel Drive,NaN,NaN,4,Other Std,U,Unknown,N,Not Available,M,Driver Front Air Bag/Passenger Active Belts,EXT,Extended Cab,N,Standard Axle,S,Single,HYD,HYDRAULIC,050,CHRYSLER,5.9L,ME,Medium Duty,R,Regular,7,Non Luxury Full Size 3qtr to 1 Ton Pickup,M,Mexico,LAGO ALBERTO,MEX,NaN,NaN,D,Domestic,5.9,V-type,OHV,Overhead Valve,2.0,16.0,Z,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,1,Alabama,10003,1,CHEV,2004.0,P,Passenger Car,CHEVROLET,MALIBU,LT,NaN,NaN,NaN,NaN,SD,Sedan,4.0,0.0,0.0,C137,GENERAL MOTORS,214.000000,0.0,6.0,NaN,G,Gas,U,Unknown,F,Fuel Injection,NaN,NaN,NaN,106.3,106.3,NaN,0.0,38,16R215,NaN,0.0,NaN,NaN,NaN,3315.0,23495.0,FWD,Front Wheel Drive,NaN,NaN,2,All Wheel Std,P,Pass Key,S,Standard,K,Dual Air Bags Front and Sides/Active Belts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,Non Luxury Traditional Mid Size,F,United States,FAIRFAX,USA,KS,KANSAS,D,Domestic,3.5,V-type,NaN,NaN,0.0,0.0,8,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,1,Alabama,10003,2,TOYT,1997.0,P,Passenger Car,TOYOTA,CAMRY,CE,LE,XLE,NaN,NaN,SD,Sedan,4.0,0.0,0.0,C175,TOYOTA,183.000000,0.0,6.0,NaN,G,Gas,U,Unknown,F,Fuel Injection,NaN,NaN,NaN,105.1,105.1,NaN,0.0,30,15R205,NaN,0.0,NaN,NaN,NaN,3086.0,19248.0,FWD,Front Wheel Drive,NaN,NaN,5,All Wheel Opt,U,Unknown,N,Not Available,E,Dual Front Air Bag/Active Belts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,Non Luxury Traditional Mid Size,U,United States,GEORGETOWN,USA,KY,KENTUCKY,B,Import Built in North America,3.0,V-type,NaN,NaN,4.0,24.0,F,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,1,Alabama,10004,1,TOYT,1999.0,T,Truck,TOYOTA,TACOMA,NaN,NaN,NaN,NaN,NaN,PK,Pickup,2.0,4.0,2.0,C175,TOYOTA,149.000000,0.0,4.0,NaN,G,Gas,U,Unknown,F,Fuel Injection,NaN,1.0,"0 - 6,000#",103.3,103.3,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,B,2567.0,12698.0,RWD,Rear Wheel Drive,NaN,NaN,5,All Wheel Opt,U,Unknown,N,Not Available,Z,Du Ar Bgs Frnt/Act Blts/Pass Deactivate/cutoff,CON,Conventional,U,Unknown,U,Unknown,U,Unknown,NaN,NaN,NaN,U,Unknown,R,Regular,T,Non Luxury Mid Size Pickup,Z,United States,FREMONT,USA,CA,CALIFORNIA,B,Import Built in North America,2.4,In-Line,DOHC,Double Overhead Camshaft,0.0,0.0,L,N,N

In [21]:
vehicles[vehicles["incomplt"]=="Y"]

,caseyear,state,statename,st_case,veh_no,ncicmake,vinyear,vehtype,vehtype_t,vinmake_t,vinmodel_t,vintrim_t,vintrim1_t,vintrim2_t,vintrim3_t,vintrim4_t,bodystyl,bodystyl_t,doors,wheels,drivwhls,mfg,mfg_t,displci,displcc,cylndrs,cycles,fuel,fuel_t,fuelinj,fuelinj_t,carbtype,carbtype_t,carbbrls,gvwrange,gvwrange_t,whlbsh,whlblg,tiredesc_f,psi_f,tiresz_f,tiresz_f_t,tiredesc_r,psi_r,rearsize,rearsize_t,tonrating,shipweight,msrp,drivetyp,drivetyp_t,salectry,salectry_t,abs,abs_t,security,security_t,drl,drl_t,rstrnt,rstrnt_t,tkcab,tkcab_t,tkaxlef,tkaxlef_t,tkaxler,tkaxler_t,tkbrak,tkbrak_t,engmfg,engmfg_t,engmodel,tkduty,tkduty_t,tkbedl,tkbedl_t,segmnt,segmnt_t,plant,plntctry_t,plntcity,plntctry,plntstat,plntstat_t,origin,origin_t,dispclmt,blocktype,enghead,enghead_t,vlvclndr,vlvtotal,engvincd,incomplt,battyp,battyp_t,batkwrtg,batvolt,supchrgr,supchrgr_t,turbo,turbo_t,engvvt,mcyusage,mcyusage_t
93,2014,1,Alabama,10058,2,INTL,1999.0,T,Truck,INTERNATIONAL,4000,4700,NaN,NaN,NaN,NaN,ST,Straight Truck,0.0,4.0,2.0,C162,NAVISTAR INTERNATIONAL,466.0,0.0,6.0,NaN,D,Diesel,NaN,NaN,NaN,NaN,NaN,6.0,"19,501 - 26,000#",128.0,254.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,RWD,Rear Wheel Drive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CMN,Medium Conventional,N,Standard Axle,S,Single,HYD,HYDRAULIC,165,INTERNATIONAL,DT466,ME,Medium Duty,NaN,NaN,9,Commercial Truck,H,United States,SPRINGFIELD,USA,OH,OHIO,D,Domestic,NaN,NaN,NaN,NaN,0.0,0.0,AA,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,2014,1,Alabama,10089,2,MACK,1993.0,T,Truck,MACK,600,MR600,NaN,NaN,NaN,NaN,ST,Straight Truck,0.0,6.0,4.0,C177,VOLVO GLOBAL TRUCK,728.0,0.0,6.0,NaN,D,Diesel,NaN,NaN,NaN,NaN,NaN,8.0,"33,000# +",NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,RWD,Rear Wheel Drive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TLO,Low Tilt,N,Standard Axle,T,Tandem,AIR,AIR,140,MACK,E7-300,HV,Heavy Duty,NaN,NaN,9,Commercial Truck,M,United States,MACUNGIE,USA,PA,PENNSYLVANIA,D,Domestic,NaN,NaN,NaN,NaN,0.0,0.0,K195,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,2014,1,Alabama,10116,1,CHEV,1992.0,T,Truck,CHEVROLET,G20,NaN,NaN,NaN,NaN,NaN,CG,Van Cargo,3.0,4.0,2.0,C137,GENERAL MOTORS,350.0,0.0,8.0,NaN,G,Gas,U,Unknown,F,Fuel Injection,NaN,2.0,"6,001 - 10,000#",110.0,125.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,C,0.0,0.0,RWD,Rear Wheel Drive,NaN,NaN,3,Rear Wheel Std,U,Unknown,N,Not Available,A,Active (Manual) Belts,VAN,Van,N,Standard Axle,S,Single,HYD,HYDRAULIC,080,GENERAL MOTORS,5.7L,ME,Medium Duty,R,Regular,8,Non Luxury Full Size 3qtr to 1 Ton Van,4,Canada,SCARBOROUGH,CAN,ON,ONTARIO,D,Domestic,5.7,V-type,OHV,Overhead Valve,2.0,16.0,K,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,2014,1,Alabama,10161,2,FORD,2014.0,T,Truck,FORD,F350,SUPER DUTY,NaN,NaN,NaN,NaN,CH,Cab Chassis,4.0,4.0,2.0,C134,FORD,378.0,0.0,8.0,NaN,G,Gas,S,Sequential,F,Fuel Injection,NaN,3.0,"10,001 - 14,000#",176.2,176.2,NaN,0.0,47,17R245,NaN,0.0,47.0,17R245,D,7023.0,32830.0,RWD,Rear Wheel Drive,NaN,NaN,2,All Wheel Std,E,Passive engine immobilizer,O,Optional,7,Du Frnt/Sd/Hd Air Bgs/Rr Hd Ar Bgs/Act Belts,CRW,Crew Cab,N,Standard Axle,S,Single,HYD,HYDRAULIC,070,FORD,6.2L,ME,Medium Duty,R,Regular,7,Non Luxury Full Size 3qtr to 1 Ton Pickup,E,United States,JEFFERSON COUNTY,USA,KY,KENTUCKY,D,Domestic,6.2,V-type,SOHC,Single Overhead Camshaft,2.0,16.0,6,Y,NaN,NaN,NaN,NaN,N,No,N,No,N,NaN,NaN
279,2014,1,Alabama,10186,2,FRHT,2011.0,T,Truck,FREIGHTLINER,M2,106 MEDIUM DUTY,NaN,NaN,NaN,NaN,ST,Straight Truck,0.0,4.0,2.0,C127,DAIMLER-CHRYSLER,505.0,0.0,6.0,NaN,D,Diesel,NaN,NaN,NaN,NaN,NaN,7.0,"26,001 - 33,000#",NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,RWD,Rear Wheel Drive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CMN,Medium Conventional,U,Unknown,S,Single,AIR,AIR,020,CUMMINS,ISC,ME,Medium Duty,NaN,NaN,9,Commercial Truck,D,Mexico,SANTIAGO,MEX,NaN,NaN,D,Domestic,8.3,NaN,NaN,NaN,0.0,0.0,BS,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

1.2) Determine the useless columns to drop from the dataframes.

**Hint 1**: An encoded and a decoded (actual-valued) column exist for most of the features. Drop the encoded columns.<br>
**Hint 2**: Some features have mostly null values or a single value. <br>
**Hint 3**: Some features provides information that can be obtained after the accident. <br>
You can refer to [FARS User’s Manual](https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/813023) if needed.

In [22]:
# Be aware that this is a mutable method.
def drop_useless_columns(vehicles,persons):
    
    vec_cols_to_drop = ['state', 'ncicmake', 'vehtype', 'vintrim_t', 'vintrim1_t', 'vintrim2_t' ,'vintrim3_t', 'vintrim4_t', 'bodystyl', 'mfg', 'displcc', 'cycles','fuel','fuelinj','fuelinj_t', 'carbtype_t','abs_t','security_t',  'carbbrls', 'gvwrange' ,'gvwrange_t' 
                        ,'whlbsh', 'whlblg' , 'tiredesc_f', 'psi_f', 'tiresz_f' ,'tiresz_f_t', 'tiredesc_r' ,'psi_r' ,'rearsize', 'rearsize_t', 'tonrating' ,'drivetyp', 'salectry' ,'salectry_t', 'abs', 'security', 'drl','rstrnt_t' ,'rstrnt'
                        , 'tkcab' ,'tkcab_t' ,'tkaxlef' ,'tkaxlef_t', 'tkaxler', 'tkaxler_t', 'tkbrak', 'tkbrak_t', 'engmfg' ,'engmfg_t', 'engmodel', 'tkduty' ,'tkduty_t' ,'tkbedl', 'tkbedl_t', 'segmnt' ,'plant'
                        , 'plntctry', 'plntstat', 'plntstat_t','origin','dispclmt', 'blocktype' ,'enghead' ,'enghead_t' , 'battyp', 'battyp_t', 'batkwrtg', 'batvolt' ,'supchrgr' ,'supchrgr_t' ,'turbo', 'turbo_t' ,'engvvt','mcyusage' ,'mcyusage_t']
    # fullfill the list with some column names of vehicles df,
    per_cols_to_drop = ["state","county","month","hour","minute","minutename","road_fnc","road_fncname","harm_ev","harm_evname","man_collname","man_coll"
                        ,"sch_bus","make","mak_mod","body_typ","mod_year","tow_veh","spec_use","emer_use","emer_usename","rollover","impact1","impact1name","fire_exp","fire_expname","agename","sex","per_typname","inj_sev","inj_sevname",
                        "seat_pos","rest_use","rest_mis","rest_misname","air_bag","air_bagname","ejection","ejectionname","ej_path","ej_pathname","extricat","extricatname","drinking","alc_det","alc_detname","alc_status","alc_statusname","atst_typ","atst_typname","alc_res"
                        ,"alc_resname","drugs","drug_det","drug_detname","dstatus","dstatusname","drugtst1","drugtst1name","drugres1","drugres1name","drugtst2","drugtst2name","drugres2","drugres2name","drugtst3"
                        ,"drugtst3name","drugres3","drugres3name","hospital","hospitalname","doa","doaname","death_da","death_daname","death_mo","death_moname","death_yr","death_yrname","death_hr","death_hrname","death_mn","death_mnname","death_tm","lag_hrs","lag_mins","p_sf1","p_sf2","p_sf3"
                        ,"cert_no","work_inj","work_injname","hispanic","race","location","locationname","func_sys","func_sysname","rur_urb","rur_urbname","racename"] 
    # fullfill the list with some column names of persons df

    vehicles.drop(vec_cols_to_drop, axis=1, inplace=True)
    persons.drop(per_cols_to_drop, axis=1, inplace=True)

In [23]:
drop_useless_columns(copy_vehicles, copy_persons)
copy_vehicles

,caseyear,statename,st_case,veh_no,vinyear,vehtype_t,vinmake_t,vinmodel_t,bodystyl_t,doors,wheels,drivwhls,mfg_t,displci,cylndrs,fuel_t,carbtype,shipweight,msrp,drivetyp_t,drl_t,segmnt_t,plntctry_t,plntcity,origin_t,vlvclndr,vlvtotal,engvincd,incomplt
0,2014,Alabama,10001,1,2011.0,Passenger Car,TOYOTA,COROLLA,Sedan,4.0,4.0,2.0,TOYOTA,110.000000,4.0,Gas,F,2734.0,15600.0,Front Wheel Drive,Standard,Non Luxury Traditional Compact,Canada,CAMBRIDGE,Import Built in North America,4.0,16.0,U,N
1,2014,Alabama,10002,1,1997.0,Truck,DODGE,RAM 2500,Pickup,2.0,4.0,2.0,DAIMLER-CHRYSLER,360.000000,8.0,Gas,F,4787.0,20775.0,Rear Wheel Drive,Not Available,Non Luxury Full Size 3qtr to 1 Ton Pickup,Mexico,LAGO ALBERTO,Domestic,2.0,16.0,Z,N
2,2014,Alabama,10003,1,2004.0,Passenger Car,CHEVROLET,MALIBU,Sedan,4.0,0.0,0.0,GENERAL MOTORS,214.000000,6.0,Gas,F,3315.0,23495.0,Front Wheel Drive,Standard,Non Luxury Traditional Mid Size,United States,FAIRFAX,Domestic,0.0,0.0,8,N
3,2014,Alabama,10003,2,1997.0,Passenger Car,TOYOTA,CAMRY,Sedan,4.0,0.0,0.0,TOYOTA,183.000000,6.0,Gas,F,3086.0,19248.0,Front Wheel Drive,Not Available,Non Luxury Traditional Mid Size,United States,GEORGETOWN,Import Built in North America,4.0,24.0,F,N
4,2014,Alabama,10004,1,1999.0,Truck,TOYOTA,TACOMA,Pickup,2.0,4.0,2.0,TOYOTA,149.000000,4.0,Gas,F,2567.0,12698.0,Rear Wheel Drive,Not Available,Non Luxury Mid Size Pickup,United States,FREMONT,Import Built in North America,0.0,0.0,L,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52987,2019,Wyoming,560118,5,2000.0,Truck,PETERBILT,379,TRACTOR TRUCK,0.0,6.0,4.0,Paccar,893.000000,6,Diesel,NaN,0.0,0.0,Rear Wheel Drive,NaN,Commercial Truck,United States,MADISON,Domestic,0.0,0.0,B,N
52988,2019,Wyoming,560119,1,2018.0,Motorcycle,HARLEY-DAVIDSON,FLHXSE,TOURING,0.0,0.0,0.0,Harley-Davidson,117.073171,2,NaN,NaN,842.0,39949.0,NaN,NaN,NaN,United States,YORK,Domestic,0.0,0.0,NaN,N
52989,2019,Wyoming,560120,1,2010.0,Truck,HYUNDAI,SANTA FE,SPORT UTILITY VEHICLE,4.0,4.0,4.0,Hyundai,212.000000,6,Gas,F,4120.0,30295.0,All Wheel Drive,Not Available,Non Luxury Mid Size CUV,United States,ALABAMA PLANT,Import Built in North America,4.0,24.0,G,N
52990,2019,Wyoming,560121,1,2013.0,Passenger Car,DODGE,DART,SEDAN,4.0,4.0,2.0,Chrysler Group LLC,122.000000,4,Gas,F,3186.0,17995.0,Front Wheel Drive,Optional,Non Luxury Traditional Compact,United States,BELVIDERE,Domestic,4.0,16.0,A,N


In [24]:
copy_persons

,caseyear,statename,st_case,ve_forms,veh_no,per_no,str_veh,str_vehname,countyname,day,monthname,hourname,sch_busname,makename,body_typname,mod_yearname,tow_vehname,spec_usename,rollovername,age,sexname,per_typ,seat_posname,rest_usename,drinkingname,drugsname,p_sf1name,p_sf2name,p_sf3name,hispanicname
0,2014,Alabama,10001,1,1,1,0,Occupant of a Motor Vehicle,JACKSON (71),1,January,1:00am-1:59am,No,Toyota,"4-door sedan, hardtop",2011,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",24,Male,1,"Front Seat, Left Side",Shoulder and Lap Belt Used,Yes (Alcohol Involved),Unknown,None,None,None,Non-Hispanic
1,2014,Alabama,10001,1,1,2,0,Occupant of a Motor Vehicle,JACKSON (71),1,January,1:00am-1:59am,No,Toyota,"4-door sedan, hardtop",2011,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",30,Female,2,"Front Seat, Right Side",Shoulder and Lap Belt Used,Not Reported,Not Reported,None,None,None,Not A Fatality (not Applicable)
2,2014,Alabama,10002,1,1,1,0,Occupant of a Motor Vehicle,FRANKLIN (59),1,January,1:00pm-1:59pm,No,Dodge,"Standard pickup (GVWR 4,500 to 10,00 lbs.)(Jee...",1997,No Trailing Units,No Special Use,No Rollover,52,Male,1,"Front Seat, Left Side",None Used,Yes (Alcohol Involved),No (drugs not involved),None,None,None,Non-Hispanic
3,2014,Alabama,10003,2,1,1,0,Occupant of a Motor Vehicle,TUSCALOOSA (125),1,January,3:00am-3:59am,No,Chevrolet,"4-door sedan, hardtop",2004,No Trailing Units,No Special Use,No Rollover,22,Male,1,"Front Seat, Left Side",None Used,Unknown (Police Reported),Unknown,None,None,None,Not A Fatality (not Applicable)
4,2014,Alabama,10003,2,1,2,0,Occupant of a Motor Vehicle,TUSCALOOSA (125),1,January,3:00am-3:59am,No,Chevrolet,"4-door sedan, hardtop",2004,No Trailing Units,No Special Use,No Rollover,21,Female,2,"Front Seat, Right Side",Shoulder and Lap Belt Used,Not Reported,Not Reported,None,None,None,Not A Fatality (not Applicable)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82879,2019,Wyoming,560120,1,1,1,0,Occupant of a Motor Vehicle,LARAMIE (21),22,September,4:00am-4:59am,No,Hyundai,"Compact Utility (Utility Vehicle Categories ""S...",2010,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",31,Male,1,"Front Seat, Left Side",None Used/Not Applicable,Yes (Alcohol Involved),Yes (drugs involved),None,None,None,Non-Hispanic
82880,2019,Wyoming,560120,1,1,2,0,Occupant of a Motor Vehicle,LARAMIE (21),22,September,4:00am-4:59am,No,Hyundai,"Compact Utility (Utility Vehicle Categories ""S...",2010,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",22,Male,2,"Front Seat, Right Side",None Used/Not Applicable,Not Reported,Not Reported,None,None,None,Not A Fatality (not Applicable)
82881,2019,Wyoming,560121,2,1,1,0,Occupant of a Motor Vehicle,SWEETWATER (37),27,September,11:00pm-11:59pm,No,Dodge,"4-door sedan, hardtop",2013,No Trailing Units,No Special Use,No Rollover,42,Male,1,"Front Seat, Left Side",Shoulder and Lap Belt Used,Yes (Alcohol Involved),No (drugs not involved),None,None,None,Non-Hispanic
82882,2019,Wyoming,560121,2,1,2,0,Occupant of a Motor Vehicle,SWEETWATER (37),27,September,11:00pm-11:59pm,No,Dodge,"4-door sedan, hardtop",2013,No Trailing Units,No Special Use,No Rollover,41,Female,2,"Front Seat, Right Side",Shoulder and Lap Belt Used,Not Reported,Not Reported,None,None,None,Non-Hispanic


In [25]:
cc_persons=copy_persons.copy()
cc2_persons=copy_persons.copy()
cc3_persons=copy_persons.copy()

In [26]:
cc_vehicles=copy_vehicles.copy()
cc2_vehicles=copy_vehicles.copy()
cc3_vehicles=copy_vehicles.copy()

In [27]:
# YOUR CODE HERE
cc_vehicles

,caseyear,statename,st_case,veh_no,vinyear,vehtype_t,vinmake_t,vinmodel_t,bodystyl_t,doors,wheels,drivwhls,mfg_t,displci,cylndrs,fuel_t,carbtype,shipweight,msrp,drivetyp_t,drl_t,segmnt_t,plntctry_t,plntcity,origin_t,vlvclndr,vlvtotal,engvincd,incomplt
0,2014,Alabama,10001,1,2011.0,Passenger Car,TOYOTA,COROLLA,Sedan,4.0,4.0,2.0,TOYOTA,110.000000,4.0,Gas,F,2734.0,15600.0,Front Wheel Drive,Standard,Non Luxury Traditional Compact,Canada,CAMBRIDGE,Import Built in North America,4.0,16.0,U,N
1,2014,Alabama,10002,1,1997.0,Truck,DODGE,RAM 2500,Pickup,2.0,4.0,2.0,DAIMLER-CHRYSLER,360.000000,8.0,Gas,F,4787.0,20775.0,Rear Wheel Drive,Not Available,Non Luxury Full Size 3qtr to 1 Ton Pickup,Mexico,LAGO ALBERTO,Domestic,2.0,16.0,Z,N
2,2014,Alabama,10003,1,2004.0,Passenger Car,CHEVROLET,MALIBU,Sedan,4.0,0.0,0.0,GENERAL MOTORS,214.000000,6.0,Gas,F,3315.0,23495.0,Front Wheel Drive,Standard,Non Luxury Traditional Mid Size,United States,FAIRFAX,Domestic,0.0,0.0,8,N
3,2014,Alabama,10003,2,1997.0,Passenger Car,TOYOTA,CAMRY,Sedan,4.0,0.0,0.0,TOYOTA,183.000000,6.0,Gas,F,3086.0,19248.0,Front Wheel Drive,Not Available,Non Luxury Traditional Mid Size,United States,GEORGETOWN,Import Built in North America,4.0,24.0,F,N
4,2014,Alabama,10004,1,1999.0,Truck,TOYOTA,TACOMA,Pickup,2.0,4.0,2.0,TOYOTA,149.000000,4.0,Gas,F,2567.0,12698.0,Rear Wheel Drive,Not Available,Non Luxury Mid Size Pickup,United States,FREMONT,Import Built in North America,0.0,0.0,L,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52987,2019,Wyoming,560118,5,2000.0,Truck,PETERBILT,379,TRACTOR TRUCK,0.0,6.0,4.0,Paccar,893.000000,6,Diesel,NaN,0.0,0.0,Rear Wheel Drive,NaN,Commercial Truck,United States,MADISON,Domestic,0.0,0.0,B,N
52988,2019,Wyoming,560119,1,2018.0,Motorcycle,HARLEY-DAVIDSON,FLHXSE,TOURING,0.0,0.0,0.0,Harley-Davidson,117.073171,2,NaN,NaN,842.0,39949.0,NaN,NaN,NaN,United States,YORK,Domestic,0.0,0.0,NaN,N
52989,2019,Wyoming,560120,1,2010.0,Truck,HYUNDAI,SANTA FE,SPORT UTILITY VEHICLE,4.0,4.0,4.0,Hyundai,212.000000,6,Gas,F,4120.0,30295.0,All Wheel Drive,Not Available,Non Luxury Mid Size CUV,United States,ALABAMA PLANT,Import Built in North America,4.0,24.0,G,N
52990,2019,Wyoming,560121,1,2013.0,Passenger Car,DODGE,DART,SEDAN,4.0,4.0,2.0,Chrysler Group LLC,122.000000,4,Gas,F,3186.0,17995.0,Front Wheel Drive,Optional,Non Luxury Traditional Compact,United States,BELVIDERE,Domestic,4.0,16.0,A,N


1.3.1) Complete the following method returning a single DataFrame named 'accidents' whose rows are singular for person and vehicle data. Then, merge the dataframes belongs to the same year.

**Hint 1**: You need to define a key from some columns like year, st_case, and veh_no to have an unique value for each row. <br>
**Hint 2**: You might use such methods ```<DataFrame>.merge```, ```<DataFrame>.join```, or ```pd.concat```.

In [28]:
def merge_vehicles_and_persons(vehicles, persons) -> pd.DataFrame:
    
    # YOUR CODE HERE
    # accidents = ...
    accidents = vehicles.merge(persons, how='inner', on= ['caseyear','st_case','veh_no','statename' ])
    
    return accidents

In [29]:
# YOUR CODE HERE
accidents=merge_vehicles_and_persons(cc2_vehicles,cc2_persons)


In [30]:
accidents

,caseyear,statename,st_case,veh_no,vinyear,vehtype_t,vinmake_t,vinmodel_t,bodystyl_t,doors,wheels,drivwhls,mfg_t,displci,cylndrs,fuel_t,carbtype,shipweight,msrp,drivetyp_t,drl_t,segmnt_t,plntctry_t,plntcity,origin_t,vlvclndr,vlvtotal,engvincd,incomplt,ve_forms,per_no,str_veh,str_vehname,countyname,day,monthname,hourname,sch_busname,makename,body_typname,mod_yearname,tow_vehname,spec_usename,rollovername,age,sexname,per_typ,seat_posname,rest_usename,drinkingname,drugsname,p_sf1name,p_sf2name,p_sf3name,hispanicname
0,2014,Alabama,10001,1,2011.0,Passenger Car,TOYOTA,COROLLA,Sedan,4.0,4.0,2.0,TOYOTA,110.0,4.0,Gas,F,2734.0,15600.0,Front Wheel Drive,Standard,Non Luxury Traditional Compact,Canada,CAMBRIDGE,Import Built in North America,4.0,16.0,U,N,1,1,0,Occupant of a Motor Vehicle,JACKSON (71),1,January,1:00am-1:59am,No,Toyota,"4-door sedan, hardtop",2011,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",24,Male,1,"Front Seat, Left Side",Shoulder and Lap Belt Used,Yes (Alcohol Involved),Unknown,None,None,None,Non-Hispanic
1,2014,Alabama,10001,1,2011.0,Passenger Car,TOYOTA,COROLLA,Sedan,4.0,4.0,2.0,TOYOTA,110.0,4.0,Gas,F,2734.0,15600.0,Front Wheel Drive,Standard,Non Luxury Traditional Compact,Canada,CAMBRIDGE,Import Built in North America,4.0,16.0,U,N,1,2,0,Occupant of a Motor Vehicle,JACKSON (71),1,January,1:00am-1:59am,No,Toyota,"4-door sedan, hardtop",2011,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",30,Female,2,"Front Seat, Right Side",Shoulder and Lap Belt Used,Not Reported,Not Reported,None,None,None,Not A Fatality (not Applicable)
2,2014,Alabama,10002,1,1997.0,Truck,DODGE,RAM 2500,Pickup,2.0,4.0,2.0,DAIMLER-CHRYSLER,360.0,8.0,Gas,F,4787.0,20775.0,Rear Wheel Drive,Not Available,Non Luxury Full Size 3qtr to 1 Ton Pickup,Mexico,LAGO ALBERTO,Domestic,2.0,16.0,Z,N,1,1,0,Occupant of a Motor Vehicle,FRANKLIN (59),1,January,1:00pm-1:59pm,No,Dodge,"Standard pickup (GVWR 4,500 to 10,00 lbs.)(Jee...",1997,No Trailing Units,No Special Use,No Rollover,52,Male,1,"Front Seat, Left Side",None Used,Yes (Alcohol Involved),No (drugs not involved),None,None,None,Non-Hispanic
3,2014,Alabama,10003,1,2004.0,Passenger Car,CHEVROLET,MALIBU,Sedan,4.0,0.0,0.0,GENERAL MOTORS,214.0,6.0,Gas,F,3315.0,23495.0,Front Wheel Drive,Standard,Non Luxury Traditional Mid Size,United States,FAIRFAX,Domestic,0.0,0.0,8,N,2,1,0,Occupant of a Motor Vehicle,TUSCALOOSA (125),1,January,3:00am-3:59am,No,Chevrolet,"4-door sedan, hardtop",2004,No Trailing Units,No Special Use,No Rollover,22,Male,1,"Front Seat, Left Side",None Used,Unknown (Police Reported),Unknown,None,None,None,Not A Fatality (not Applicable)
4,2014,Alabama,10003,1,2004.0,Passenger Car,CHEVROLET,MALIBU,Sedan,4.0,0.0,0.0,GENERAL MOTORS,214.0,6.0,Gas,F,3315.0,23495.0,Front Wheel Drive,Standard,Non Luxury Traditional Mid Size,United States,FAIRFAX,Domestic,0.0,0.0,8,N,2,2,0,Occupant of a Motor Vehicle,TUSCALOOSA (125),1,January,3:00am-3:59am,No,Chevrolet,"4-door sedan, hardtop",2004,No Trailing Units,No Special Use,No Rollover,21,Female,2,"Front Seat, Right Side",Shoulder and Lap Belt Used,Not Reported,Not Reported,None,None,None,Not A Fatality (not Applicable)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452905,2019,Wyoming,560120,1,2010.0,Truck,HYUNDAI,SANTA FE,SPORT UTILITY VEHICLE,4.0,4.0,4.0,Hyundai,212.0,6,Gas,F,4120.0,30295.0,All Wheel Drive,Not Available,Non Luxury Mid Size CUV,United States,ALABAMA PLANT,Import Built in North America,4.0,24.0,G,N,1,1,0,Occupant of a Motor Vehicle,LARAMIE (21),22,September,4:00am-4:59am,No,Hyundai,"Compact Utility (Utility Vehicle Categories ""S...",2010,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",31,Male,1,"Front Seat, Left Side",None Used/Not Applicable,Yes (Alcohol Involved),Yes (drugs involved),None,None,None,Non-Hispan

1.3.2) Obtain a single dataframe named 'data' by merging accidents dataframes of all available years. There might be some columns which do not present in all years. <br> 
Save this dataframe as a csv or xlsx file so that after having the final data you can skip the previous steps while doing the assignment.


In [31]:
# YOUR CODE HERE
for i in range(50):
    print(accidents.columns[i], ": ",accidents.isnull().sum()[i])

caseyear :  0
statename :  0
st_case :  0
veh_no :  0
vinyear :  15705
vehtype_t :  15705
vinmake_t :  15705
vinmodel_t :  15729
bodystyl_t :  15705
doors :  15705
wheels :  15705
drivwhls :  15705
mfg_t :  26466
displci :  15705
cylndrs :  16418
fuel_t :  50056
carbtype :  81313
shipweight :  15705
msrp :  15705
drivetyp_t :  50501
drl_t :  90162
segmnt_t :  49973
plntctry_t :  20400
plntcity :  26003
origin_t :  16499
vlvclndr :  15705
vlvtotal :  15705
engvincd :  43976
incomplt :  15705
ve_forms :  0
per_no :  0
str_veh :  0
str_vehname :  0
countyname :  0
day :  0
monthname :  0
hourname :  0
sch_busname :  0
makename :  0
body_typname :  0
mod_yearname :  0
tow_vehname :  0
spec_usename :  0
rollovername :  0
age :  0
sexname :  0
per_typ :  0
seat_posname :  0
rest_usename :  0
drinkingname :  0


In [32]:
accidents.to_csv('FARS.csv')

In [33]:
# Read the saved final data file as a DataFrame.
# Remark: Your final dataset should comprise approximately 60 columns. 

data = pd.read_csv('FARS.csv')

In [34]:
data

,Unnamed: 0,caseyear,statename,st_case,veh_no,vinyear,vehtype_t,vinmake_t,vinmodel_t,bodystyl_t,doors,wheels,drivwhls,mfg_t,displci,cylndrs,fuel_t,carbtype,shipweight,msrp,drivetyp_t,drl_t,segmnt_t,plntctry_t,plntcity,origin_t,vlvclndr,vlvtotal,engvincd,incomplt,ve_forms,per_no,str_veh,str_vehname,countyname,day,monthname,hourname,sch_busname,makename,body_typname,mod_yearname,tow_vehname,spec_usename,rollovername,age,sexname,per_typ,seat_posname,rest_usename,drinkingname,drugsname,p_sf1name,p_sf2name,p_sf3name,hispanicname
0,0,2014,Alabama,10001,1,2011.0,Passenger Car,TOYOTA,COROLLA,Sedan,4.0,4.0,2.0,TOYOTA,110.0,4.0,Gas,F,2734.0,15600.0,Front Wheel Drive,Standard,Non Luxury Traditional Compact,Canada,CAMBRIDGE,Import Built in North America,4.0,16.0,U,N,1,1,0,Occupant of a Motor Vehicle,JACKSON (71),1,January,1:00am-1:59am,No,Toyota,"4-door sedan, hardtop",2011,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",24,Male,1,"Front Seat, Left Side",Shoulder and Lap Belt Used,Yes (Alcohol Involved),Unknown,None,None,None,Non-Hispanic
1,1,2014,Alabama,10001,1,2011.0,Passenger Car,TOYOTA,COROLLA,Sedan,4.0,4.0,2.0,TOYOTA,110.0,4.0,Gas,F,2734.0,15600.0,Front Wheel Drive,Standard,Non Luxury Traditional Compact,Canada,CAMBRIDGE,Import Built in North America,4.0,16.0,U,N,1,2,0,Occupant of a Motor Vehicle,JACKSON (71),1,January,1:00am-1:59am,No,Toyota,"4-door sedan, hardtop",2011,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",30,Female,2,"Front Seat, Right Side",Shoulder and Lap Belt Used,Not Reported,Not Reported,None,None,None,Not A Fatality (not Applicable)
2,2,2014,Alabama,10002,1,1997.0,Truck,DODGE,RAM 2500,Pickup,2.0,4.0,2.0,DAIMLER-CHRYSLER,360.0,8.0,Gas,F,4787.0,20775.0,Rear Wheel Drive,Not Available,Non Luxury Full Size 3qtr to 1 Ton Pickup,Mexico,LAGO ALBERTO,Domestic,2.0,16.0,Z,N,1,1,0,Occupant of a Motor Vehicle,FRANKLIN (59),1,January,1:00pm-1:59pm,No,Dodge,"Standard pickup (GVWR 4,500 to 10,00 lbs.)(Jee...",1997,No Trailing Units,No Special Use,No Rollover,52,Male,1,"Front Seat, Left Side",None Used,Yes (Alcohol Involved),No (drugs not involved),None,None,None,Non-Hispanic
3,3,2014,Alabama,10003,1,2004.0,Passenger Car,CHEVROLET,MALIBU,Sedan,4.0,0.0,0.0,GENERAL MOTORS,214.0,6.0,Gas,F,3315.0,23495.0,Front Wheel Drive,Standard,Non Luxury Traditional Mid Size,United States,FAIRFAX,Domestic,0.0,0.0,8,N,2,1,0,Occupant of a Motor Vehicle,TUSCALOOSA (125),1,January,3:00am-3:59am,No,Chevrolet,"4-door sedan, hardtop",2004,No Trailing Units,No Special Use,No Rollover,22,Male,1,"Front Seat, Left Side",None Used,Unknown (Police Reported),Unknown,None,None,None,Not A Fatality (not Applicable)
4,4,2014,Alabama,10003,1,2004.0,Passenger Car,CHEVROLET,MALIBU,Sedan,4.0,0.0,0.0,GENERAL MOTORS,214.0,6.0,Gas,F,3315.0,23495.0,Front Wheel Drive,Standard,Non Luxury Traditional Mid Size,United States,FAIRFAX,Domestic,0.0,0.0,8,N,2,2,0,Occupant of a Motor Vehicle,TUSCALOOSA (125),1,January,3:00am-3:59am,No,Chevrolet,"4-door sedan, hardtop",2004,No Trailing Units,No Special Use,No Rollover,21,Female,2,"Front Seat, Right Side",Shoulder and Lap Belt Used,Not Reported,Not Reported,None,None,None,Not A Fatality (not Applicable)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452905,452905,2019,Wyoming,560120,1,2010.0,Truck,HYUNDAI,SANTA FE,SPORT UTILITY VEHICLE,4.0,4.0,4.0,Hyundai,212.0,6,Gas,F,4120.0,30295.0,All Wheel Drive,Not Available,Non Luxury Mid Size CUV,United States,ALABAMA PLANT,Import Built in North America,4.0,24.0,G,N,1,1,0,Occupant of a Motor Vehicle,LARAMIE (21),22,September,4:00am-4:59am,No,Hyundai,"Compact Utility (Utility Vehicle Categories ""S...",2010,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",31,Male,1,"Front Seat, Left Side",None Used/Not Applicable,Yes (Alcohol Involved),Yes (drugs inv

In [35]:
data=data.drop(['Unnamed: 0'], axis=1)

In [36]:
data

,caseyear,statename,st_case,veh_no,vinyear,vehtype_t,vinmake_t,vinmodel_t,bodystyl_t,doors,wheels,drivwhls,mfg_t,displci,cylndrs,fuel_t,carbtype,shipweight,msrp,drivetyp_t,drl_t,segmnt_t,plntctry_t,plntcity,origin_t,vlvclndr,vlvtotal,engvincd,incomplt,ve_forms,per_no,str_veh,str_vehname,countyname,day,monthname,hourname,sch_busname,makename,body_typname,mod_yearname,tow_vehname,spec_usename,rollovername,age,sexname,per_typ,seat_posname,rest_usename,drinkingname,drugsname,p_sf1name,p_sf2name,p_sf3name,hispanicname
0,2014,Alabama,10001,1,2011.0,Passenger Car,TOYOTA,COROLLA,Sedan,4.0,4.0,2.0,TOYOTA,110.0,4.0,Gas,F,2734.0,15600.0,Front Wheel Drive,Standard,Non Luxury Traditional Compact,Canada,CAMBRIDGE,Import Built in North America,4.0,16.0,U,N,1,1,0,Occupant of a Motor Vehicle,JACKSON (71),1,January,1:00am-1:59am,No,Toyota,"4-door sedan, hardtop",2011,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",24,Male,1,"Front Seat, Left Side",Shoulder and Lap Belt Used,Yes (Alcohol Involved),Unknown,None,None,None,Non-Hispanic
1,2014,Alabama,10001,1,2011.0,Passenger Car,TOYOTA,COROLLA,Sedan,4.0,4.0,2.0,TOYOTA,110.0,4.0,Gas,F,2734.0,15600.0,Front Wheel Drive,Standard,Non Luxury Traditional Compact,Canada,CAMBRIDGE,Import Built in North America,4.0,16.0,U,N,1,2,0,Occupant of a Motor Vehicle,JACKSON (71),1,January,1:00am-1:59am,No,Toyota,"4-door sedan, hardtop",2011,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",30,Female,2,"Front Seat, Right Side",Shoulder and Lap Belt Used,Not Reported,Not Reported,None,None,None,Not A Fatality (not Applicable)
2,2014,Alabama,10002,1,1997.0,Truck,DODGE,RAM 2500,Pickup,2.0,4.0,2.0,DAIMLER-CHRYSLER,360.0,8.0,Gas,F,4787.0,20775.0,Rear Wheel Drive,Not Available,Non Luxury Full Size 3qtr to 1 Ton Pickup,Mexico,LAGO ALBERTO,Domestic,2.0,16.0,Z,N,1,1,0,Occupant of a Motor Vehicle,FRANKLIN (59),1,January,1:00pm-1:59pm,No,Dodge,"Standard pickup (GVWR 4,500 to 10,00 lbs.)(Jee...",1997,No Trailing Units,No Special Use,No Rollover,52,Male,1,"Front Seat, Left Side",None Used,Yes (Alcohol Involved),No (drugs not involved),None,None,None,Non-Hispanic
3,2014,Alabama,10003,1,2004.0,Passenger Car,CHEVROLET,MALIBU,Sedan,4.0,0.0,0.0,GENERAL MOTORS,214.0,6.0,Gas,F,3315.0,23495.0,Front Wheel Drive,Standard,Non Luxury Traditional Mid Size,United States,FAIRFAX,Domestic,0.0,0.0,8,N,2,1,0,Occupant of a Motor Vehicle,TUSCALOOSA (125),1,January,3:00am-3:59am,No,Chevrolet,"4-door sedan, hardtop",2004,No Trailing Units,No Special Use,No Rollover,22,Male,1,"Front Seat, Left Side",None Used,Unknown (Police Reported),Unknown,None,None,None,Not A Fatality (not Applicable)
4,2014,Alabama,10003,1,2004.0,Passenger Car,CHEVROLET,MALIBU,Sedan,4.0,0.0,0.0,GENERAL MOTORS,214.0,6.0,Gas,F,3315.0,23495.0,Front Wheel Drive,Standard,Non Luxury Traditional Mid Size,United States,FAIRFAX,Domestic,0.0,0.0,8,N,2,2,0,Occupant of a Motor Vehicle,TUSCALOOSA (125),1,January,3:00am-3:59am,No,Chevrolet,"4-door sedan, hardtop",2004,No Trailing Units,No Special Use,No Rollover,21,Female,2,"Front Seat, Right Side",Shoulder and Lap Belt Used,Not Reported,Not Reported,None,None,None,Not A Fatality (not Applicable)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452905,2019,Wyoming,560120,1,2010.0,Truck,HYUNDAI,SANTA FE,SPORT UTILITY VEHICLE,4.0,4.0,4.0,Hyundai,212.0,6,Gas,F,4120.0,30295.0,All Wheel Drive,Not Available,Non Luxury Mid Size CUV,United States,ALABAMA PLANT,Import Built in North America,4.0,24.0,G,N,1,1,0,Occupant of a Motor Vehicle,LARAMIE (21),22,September,4:00am-4:59am,No,Hyundai,"Compact Utility (Utility Vehicle Categories ""S...",2010,No Trailing Units,No Special Use,"Rollover, Tripped by Object/Vehicle",31,Male,1,"Front Seat, Left Side",None Used/Not Applicable,Yes (Alcohol Involved),Yes (drugs involved),None,None,None,Non-Hispan

In [37]:
sorted(data.columns)

['age',
 'body_typname',
 'bodystyl_t',
 'carbtype',
 'caseyear',
 'countyname',
 'cylndrs',
 'day',
 'displci',
 'doors',
 'drinkingname',
 'drivetyp_t',
 'drivwhls',
 'drl_t',
 'drugsname',
 'engvincd',
 'fuel_t',
 'hispanicname',
 'hourname',
 'incomplt',
 'makename',
 'mfg_t',
 'mod_yearname',
 'monthname',
 'msrp',
 'origin_t',
 'p_sf1name',
 'p_sf2name',
 'p_sf3name',
 'per_no',
 'per_typ',
 'plntcity',
 'plntctry_t',
 'rest_usename',
 'rollovername',
 'sch_busname',
 'seat_posname',
 'segmnt_t',
 'sexname',
 'shipweight',
 'spec_usename',
 'st_case',
 'statename',
 'str_veh',
 'str_vehname',
 'tow_vehname',
 've_forms',
 'veh_no',
 'vehtype_t',
 'vinmake_t',
 'vinmodel_t',
 'vinyear',
 'vlvclndr',
 'vlvtotal',
 'wheels']